# Initialization

In [77]:
import imp
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
import numpy as np
#from data_parser import DataParser
from pull_data import Pull
import os
from sklearn.metrics import confusion_matrix
from prettytable import PrettyTable
from statistics import mean
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score

#from scikit_IsolatedForest import RandomForest
from sklearn.ensemble import RandomForestClassifier
#from scikit_LOFNovelty import AdaBoost
from sklearn.ensemble import AdaBoostClassifier
#from scikit_OneClassSVM import GradientBoosting
from sklearn.ensemble import GradientBoostingClassifier

In [2]:
IKEA_APP = "dev-annotated-datasets/ikea-app"
IKEA_HOMEKIT = "dev-annotated-datasets/ikea-homekit"
IP_CAM = "dev-annotated-datasets/ipcam"
NORMAL_USER = "dev-annotated-datasets/normal-user"
VOICE_ASSISTANT = "dev-annotated-datasets/voice-assistant"

# Function Definitions

In [53]:
def runModel(models):
    for key, model in models.items():
        print("### Model Name:",key," ###")
        kf = KFold(5, True)
        iteration_cnt = 0
        for train_index, test_index in kf.split(c_data,c_target):
            iteration_cnt += 1
            #Train
            model.fit(c_data[train_index],c_target[train_index])
            #Evaluate 
            p_class = model.predict(c_data[test_index])
            y_pred_valid = model.predict_proba(c_data[test_index])
            #print(p_class)

# Pull Datasets

In [106]:

a = Pull(VOICE_ASSISTANT +"/train/",1)
b = Pull(IKEA_APP +"/train/",2)
c = Pull(IKEA_HOMEKIT +"/train/",2)
d = Pull(IP_CAM +"/train/",3)
e = Pull(NORMAL_USER +"/train/",4)
c_data = np.array(a.data + b.data + c.data + d.data + e.data)
c_target = np.array(a.labels + b.labels + c.labels + d.labels + e.labels)
print("Input Data:",len(c_data)," Input Labels:",len(c_target))
print("Number of features:",a.features_cnt)

Input Data: 2214  Input Labels: 2214
Number of features: 107


# MODELS

In [60]:
MODELS = {}
MODELS["RandomForest"] = {}
MODELS["AdaBoost"] = {}
MODELS["GradientBoosting"] = {}
rng = np.random.RandomState(12345)

MODELS["RandomForest"]["RF1"] = RandomForestClassifier(random_state=rng,n_estimators=10)
MODELS["AB1"] = AdaBoostClassifier(random_state=rng)
MODELS["GradientBoosting"] = GradientBoostingClassifier(random_state=rng)


In [98]:
#runModel(MODELS["RandomForest"])

# Single Models

## Random Forest

In [107]:
rng = np.random.RandomState(12345)
model = RandomForestClassifier(random_state=rng,n_estimators=10)

kf = KFold(3, True)
iteration_cnt = 0
for train_index, test_index in kf.split(c_data,c_target):
    iteration_cnt += 1
    #Train
    model.fit(c_data[train_index],c_target[train_index])
    #Evaluate 
    y_class = model.predict(c_data[test_index])
    y_pred = model.predict_proba(c_data[test_index])
    #print(metrics.accuracy_score(y_class,c_target[test_index]))
    print(classification_report(y_class,c_target[test_index],output_dict=False))

print(cross_val_score(model, c_data, c_target, cv=3))


              precision    recall  f1-score   support

           1       0.89      0.92      0.91       115
           2       1.00      0.94      0.97       189
           3       0.92      0.93      0.92        99
           4       0.96      0.98      0.97       335

   micro avg       0.95      0.95      0.95       738
   macro avg       0.94      0.94      0.94       738
weighted avg       0.95      0.95      0.95       738

              precision    recall  f1-score   support

           1       0.84      0.91      0.87        97
           2       0.99      0.93      0.96       192
           3       0.93      0.90      0.92       126
           4       0.96      0.98      0.97       323

   micro avg       0.95      0.95      0.95       738
   macro avg       0.93      0.93      0.93       738
weighted avg       0.95      0.95      0.95       738

              precision    recall  f1-score   support

           1       0.88      0.96      0.92        99
           2       0.

## AdaBoost

In [108]:
rng = np.random.RandomState(12345)
model = AdaBoostClassifier(random_state=rng)

kf = KFold(3, True)
iteration_cnt = 0
for train_index, test_index in kf.split(c_data,c_target):
    iteration_cnt += 1
    #Train
    model.fit(c_data[train_index],c_target[train_index])
    #Evaluate 
    y_class = model.predict(c_data[test_index])
    y_pred = model.predict_proba(c_data[test_index])
    #print(metrics.accuracy_score(y_class,c_target[test_index]))
    print(classification_report(y_class,c_target[test_index],output_dict=False))

print(cross_val_score(model, c_data, c_target, cv=3))

              precision    recall  f1-score   support

           1       0.70      0.77      0.73       101
           2       0.86      0.59      0.70       275
           3       0.86      0.84      0.85       103
           4       0.63      0.82      0.71       259

   micro avg       0.73      0.73      0.73       738
   macro avg       0.76      0.76      0.75       738
weighted avg       0.76      0.73      0.73       738

              precision    recall  f1-score   support

           1       0.70      0.45      0.54       173
           2       0.60      0.73      0.66       151
           3       0.67      0.82      0.73        88
           4       0.79      0.81      0.80       326

   micro avg       0.71      0.71      0.71       738
   macro avg       0.69      0.70      0.68       738
weighted avg       0.71      0.71      0.70       738

              precision    recall  f1-score   support

           1       0.41      0.51      0.45        89
           2       0.

## GradientBoosting

In [109]:
rng = np.random.RandomState(12345)
model = GradientBoostingClassifier(random_state=rng)

kf = KFold(3, True)
iteration_cnt = 0
for train_index, test_index in kf.split(c_data,c_target):
    iteration_cnt += 1
    #Train
    model.fit(c_data[train_index],c_target[train_index])
    #Evaluate 
    y_class = model.predict(c_data[test_index])
    y_pred = model.predict_proba(c_data[test_index])
    #print(metrics.accuracy_score(y_class,c_target[test_index]))
    print(classification_report(y_class,c_target[test_index],output_dict=False))

print(cross_val_score(model, c_data, c_target, cv=3))

              precision    recall  f1-score   support

           1       0.85      0.96      0.90        92
           2       1.00      0.94      0.97       180
           3       0.95      0.92      0.93       118
           4       0.97      0.99      0.98       348

   micro avg       0.96      0.96      0.96       738
   macro avg       0.94      0.95      0.95       738
weighted avg       0.96      0.96      0.96       738

              precision    recall  f1-score   support

           1       0.84      0.94      0.89       102
           2       0.98      0.91      0.95       186
           3       0.93      0.90      0.91       112
           4       0.96      0.97      0.96       338

   micro avg       0.94      0.94      0.94       738
   macro avg       0.93      0.93      0.93       738
weighted avg       0.94      0.94      0.94       738

              precision    recall  f1-score   support

           1       0.83      1.00      0.91        96
           2       0.